In [55]:
# Assignment 0
# Each one of the datasets has properties which makes
# them hard to learn. Motivate which of the three problems is most
# difficult for a decision tree algorithm to learn.

In [56]:
# MONK-1 it is hard to choose a parameter to split between a1 and a2.
# Would require a split on all values of a1 and then a split on values of a2 (2 + 3 + 3^2 branches)

# MONK-3 the trickiest. In order to evaluate if the statement holds, you always have to check all 6 params.
# It is only possible to split the space based on all 6 parameter values. It is hard to compute the information gain
# and choose the split parameter, the output(if condition holds or not +/-) of a sample changes depending on the value
# of the remaining parameters

# MONK-3 has the lowest number of training samples and contains additional noise

In [57]:
# Assignment 1
# The file dtree.py defines a function entropy which
# calculates the entropy of a dataset. Import this file along with the
# monks datasets and use it to calculate the entropy of the training
# datasets.

In [58]:
import sys
sys.path.append('../dectrees/python')
import monkdata as m
import dtree as dt
import drawtree_qt5 as qt
import statistics
import pyqtgraph as pg

In [59]:
monks = [m.monk1, m.monk2, m.monk3]

for idx,monk in enumerate(monks):
    print("Entropy of monk" + str(idx+1) + ": " + str(dt.entropy(monk)))

Entropy of monk1: 1.0
Entropy of monk2: 0.957117428264771
Entropy of monk3: 0.9998061328047111


In [60]:
# Assignment 2
# Explain entropy for a uniform distribution and a
# non-uniform distribution, present some example distributions with
# high and low entropy.

In [61]:
# In a Uniform Distribution every element of the sample space is
# equally likely to occur/be drawn.
# Entropy is a measure of uncertainty. How uncertain are you about the
# category/outcome of a sample you would draw from a given dataset

# Coin toss/ 2 cards - 2 possibilities, equally likely.
# As uncertain as you can be about the outcome of a toss -> max Entropy

# In a non-unform distribution some elements are more likely to
# occur than others. If you make a random guess/draw, you are more likely
# to get one category over the others. 

# 3 cards - 2 black one red, in a random draw 67% that you pick a black one
# 33% red one. Not so uncertain and thus entropy is lower
# -2/3 * log(2/3) - 1/3 * log(1/3) = 0.28
# -3/4 * log(3/4) - 1/4 * log(1/4) = 0.24
# (to show the decrease against 2 red to black)

In [62]:
# Assignment 3
# Use the function averageGain (defined in dtree.py)
# to calculate the expected information gain corresponding to each of
# the six attributes. Note that the attributes are represented as
# instances of the class Attribute (defined in monkdata.py) which you
# can access via m.attributes[0], ..., m.attributes[5]. Based on
# the results, which attribute should be used for splitting
# the examples at the root node?

In [63]:
print("         ", end='')
for attribute in m.attributes:
    print(str(attribute) + "    ", end='')
print()

for idx, monk in enumerate(monks):
    print("MONK-" + str(idx+1) + " ", end='')
    for attribute in m.attributes:
        print("%.3f" % dt.averageGain(monk,attribute) + " ", end='')
    print()

         A1    A2    A3    A4    A5    A6    
MONK-1 0.075 0.006 0.005 0.026 0.287 0.001 
MONK-2 0.004 0.002 0.001 0.016 0.017 0.006 
MONK-3 0.007 0.294 0.001 0.003 0.256 0.007 


In [64]:
# Best-split attributes
# MONK-1: A5
# MONK-2: A5
# MONK-3: A2

In [65]:
# Assignment 4
# For splitting we choose the attribute that maximizes
# the information gain, Eq.3. Looking at Eq.3 how does the entropy of
# the subsets, S k , look like when the information gain is maximized?
# How can we motivate using the information gain as a heuristic for
# picking an attribute for splitting? Think about reduction in entropy
# after the split and what the entropy implies.

In [66]:
# If we split a dataset on a parameter that maximizes the information gain, 
# we minimize the entropy of the dataset. The goal is to minimize
# the entropy of a dataset, reduce the impurity/order the set.
# Therefore infromation gain as a heuristic for choosing split
# parameters exactly fits into the idea of ordering of the dataset

In [67]:
# For the monk1 data draw the decision tree up to the first two levels and
# assign the majority class of the subsets that resulted from the two splits
# to the leaf nodes.

In [68]:
print(dt.buildTree(m.monk1, m.attributes, 2))

A5(+A4(---)A6(--)A1(--+))


In [69]:
def buildTreeCustom(dataset, depth):
    if (depth > 0):
        bestAttr = dt.bestAttribute(dataset, m.attributes)
        print(str(bestAttr), end='')
        
        # Select datasets splits for each value of the bestAttr
        splits = []
        for value in bestAttr.values:
            splits.append(dt.select(dataset, bestAttr, value))
                
        for split in splits:
            # If entropy of the split > 0, the split is impure and we can further split it. Recursive call with reduced depth
            if (dt.entropy(split) > 0):
                buildTreeCustom(split, depth-1)
            else:
                print('+' if dt.mostCommon(split) else '-', end='')
    else:
        print('+' if dt.mostCommon(dataset) else '-', end='')

buildTreeCustom(m.monk1, 2)

A5+A4---A6--A1--+

In [70]:
# Assignment 5
# Build the full decision trees for all three Monk datasets using
# buildTree. Then, use the function check to measure the performance of the decision tree on both the training and
# test datasets. 
# Compute the train and test set errors for the three Monk datasets
# for the full trees. Were your assumptions about the datasets correct?
# Explain the results you get for the training and test datasets.

In [71]:
t1 = dt.buildTree(m.monk1, m.attributes)
t2 = dt.buildTree(m.monk2, m.attributes)
t3 = dt.buildTree(m.monk3, m.attributes)

print("MONK-1 Training set: " + str(dt.check(t1, m.monk1)) + ", Test set: " + str(dt.check(t1, m.monk1test)))
print("MONK-2 Training set: " + str(dt.check(t2, m.monk2)) + ", Test set: " + str(dt.check(t2, m.monk2test)))
print("MONK-3 Training set: " + str(dt.check(t3, m.monk3)) + ", Test set: " + str(dt.check(t3, m.monk3test)))

# Assumptions seem to be correct. All samples in the training datasets have been correctly classified.
# In the test sets respectively 83%, 69% and 94% correctly classified.

MONK-1 Training set: 1.0, Test set: 0.8287037037037037
MONK-2 Training set: 1.0, Test set: 0.6921296296296297
MONK-3 Training set: 1.0, Test set: 0.9444444444444444


In [72]:
import random
def partition(data, fraction):
    ldata = list(data)
    random.shuffle(ldata)
    breakPoint = int(len(ldata) * fraction)
    return ldata[:breakPoint], ldata[breakPoint:]

In [73]:
# Write code which performs the complete pruning by repeatedly calling
# allPruned and picking the tree which gives the best classification
# performance on the validation dataset. You should stop pruning when
# all the pruned trees perform worse than the current candidate.

In [74]:
def prune(tree, valSet):
    currentTree = tree
    currentPerf = dt.check(currentTree, valSet)
    pTrees = dt.allPruned(currentTree)
    for pTree in pTrees:
        if (dt.check(pTree, valSet) > currentPerf):
            currentTree = prune(pTree, valSet)
            currentPerf = dt.check(currentTree, valSet)
    return currentTree

In [75]:
monk1train, monk1val = partition(m.monk1, 0.6)
qt.drawTree(prune(t1, monk1val))

SystemExit: 0

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [76]:
# Assignment 6
# Explain pruning from a bias variance trade-off perspective.

In [77]:
# The more levels in a tree, the more accurate classification
# Thus, if we go deeper in a tree, we reduce the bias but increase
# variance (even a small change in training data, will result in a
# big change in the tree).
# Pruning increases bias as samples are not so precisely classified
# anymore due to the majority class of subtree but at the same time
# reduces variance. Outcome based on majority class not likely to
# change unless a more significant change in the training data occurs.

In [78]:
# Assignment 7
# Evaluate the effect pruning has on the test error for
# the monk1 and monk3 datasets, in particular determine the optimal
# partition into training and pruning by optimizing the parameter
# fraction. Plot the classification error on the test sets as a
# function of the parameter fraction ∈ {0.3, 0.4, 0.5, 0.6, 0.7, 0.8}.
# Note that the split of the data is random. We therefore need to
# compute the statistics over several runs of the split to be able to
# draw any conclusions. Reasonable statistics includes mean and a
# measure of the spread. Do remember to print axes labels, legends
# and data points as you will not pass without them.

In [79]:
fractions = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
m1performance = []
m3performance = []

for fraction in fractions:
    val1 = 0
    val3 = 0

    for i in range(100):
        monk1train, monk1val = partition(m.monk1, fraction)
        t1 = dt.buildTree(monk1train, m.attributes)
        val1 += dt.check(prune(t1, monk1val), m.monk1test)

        monk3train, monk3val = partition(m.monk3, fraction)
        t3 = dt.buildTree(monk3train, m.attributes)
        val3 += dt.check(prune(t3, monk3val), m.monk3test)

    val1 /= 100
    m1performance.append(val1)

    val3 /= 100
    m3performance.append(val3)

In [80]:
m1error = []
m3error = []
for i in range (0,6):
    m1error.append(1-m1performance[i])
    m3error.append(1-m3performance[i])

In [81]:
print("Errors for pruned trees for different validation sets")
print("  ", end='')

for fraction in fractions:
    print(str(fraction) + "    ", end='')
print()

for error in m1error:
    print("%.4f" %  error + " ", end='')
print() 
    
for error in m3error:
    print("%.4f" %  error + " ", end='')
print("\n")


print("Mean and Variance of error for pruned trees")
print("MONK-1 Mean error: " + str("%.4f" % statistics.mean(m1error)) + " Variance: " + str("%.4f" % statistics.variance(m1error)))
print("MONK-3 Mean error: " + str("%.4f" % statistics.mean(m3error)) + " Variance: " + str("%.4f" % statistics.variance(m3error)))
print()

print("Error for unpruned trees")
print("MONK-1: " + str("%.4f" % (1-dt.check(t1, m.monk1test))))
print("MONK-3: " + str("%.4f" % (1-dt.check(t3, m.monk3test))))

Errors for pruned trees for different validation sets
  0.3    0.4    0.5    0.6    0.7    0.8    
0.2221 0.1991 0.1758 0.1615 0.1476 0.1359 
0.0875 0.0611 0.0463 0.0430 0.0422 0.0494 

Mean and Variance of error for pruned trees
MONK-1 Mean error: 0.1737 Variance: 0.0010
MONK-3 Mean error: 0.0549 Variance: 0.0003

Error for unpruned trees
MONK-1: 0.0278
MONK-3: 0.0324


In [83]:
app = pg.mkQApp()

plt = pg.plot(fractions, m1error, title='Mean error for MONK-1', symbol='o', pen=None, left='Classification error', bottom='Fraction of the training set used for training')
plt.showGrid(x=True,y=True)

plt2 = pg.plot(fractions, m3error, title='Mean error for MONK-3', symbol='o', pen=None, left='Classification error', bottom='Fraction of the training set used for training')
plt2.showGrid(x=True,y=True)

status = app.exec_()
sys.exit(status)

SystemExit: 0

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
